# Objective

This notebook use a kaggle open data-set in order to develop an experiment in order to manage unbalanced data for a classification problem. The name of the open-data set is [Health insurance cross sell prediction](https://www.kaggle.com/datasets/anmolkumar/health-insurance-cross-sell-prediction).

## Data-set context

Our client is an Insurance company that has provided Health Insurance to its customers now they need your help in building a model to predict whether the policyholders (customers) from past year will also be interested in Vehicle Insurance provided by the company.

An insurance policy is an arrangement by which a company undertakes to provide a guarantee of compensation for specified loss, damage, illness, or death in return for the payment of a specified premium. A premium is a sum of money that the customer needs to pay regularly to an insurance company for this guarantee.

For example, you may pay a premium of Rs. 5000 each year for a health insurance cover of Rs. 200,000/- so that if, God forbid, you fall ill and need to be hospitalised in that year, the insurance provider company will bear the cost of hospitalisation etc. for upto Rs. 200,000. Now if you are wondering how can company bear such high hospitalisation cost when it charges a premium of only Rs. 5000/-, that is where the concept of probabilities comes in picture. For example, like you, there may be 100 customers who would be paying a premium of Rs. 5000 every year, but only a few of them (say 2-3) would get hospitalised that year and not everyone. This way everyone shares the risk of everyone else.

Just like medical insurance, there is vehicle insurance where every year customer needs to pay a premium of certain amount to insurance provider company so that in case of unfortunate accident by the vehicle, the insurance provider company will provide a compensation (called ‘sum assured’) to the customer.

Building a model to predict whether a customer would be interested in Vehicle Insurance is extremely helpful for the company because it can then accordingly plan its communication strategy to reach out to those customers and optimise its business model and revenue.

Now, in order to predict, whether the customer would be interested in Vehicle insurance, you have information about demographics (gender, age, region code type), Vehicles (Vehicle Age, Damage), Policy (Premium, sourcing channel) etc.

In [13]:
import pandas as pd
import numpy as np
from utilities import ut_standard_col_name

# Pre-processing tools
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder

# Tools to over-sample
from imblearn.over_sampling import SMOTENC

# Model tools
from sklearn.model_selection import train_test_split, GridSearchCV
from xgboost import XGBClassifier
from sklearn.pipeline import Pipeline

# Model evaluation tools
from sklearn.metrics import classification_report

In [2]:
# import input (training) data
input_df = pd.read_csv("/Users/lorenzopusateri/Documents/01_studio/04_kaggle/Kaggle/competitions/insurance_cross_selling/data/train.csv")
# standardize column names
input_df = ut_standard_col_name(input_df)
input_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 381109 entries, 0 to 381108
Data columns (total 12 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   id                    381109 non-null  int64  
 1   gender                381109 non-null  object 
 2   age                   381109 non-null  int64  
 3   driving_license       381109 non-null  int64  
 4   region_code           381109 non-null  float64
 5   previously_insured    381109 non-null  int64  
 6   vehicle_age           381109 non-null  object 
 7   vehicle_damage        381109 non-null  object 
 8   annual_premium        381109 non-null  float64
 9   policy_sales_channel  381109 non-null  float64
 10  vintage               381109 non-null  int64  
 11  response              381109 non-null  int64  
dtypes: float64(3), int64(6), object(3)
memory usage: 34.9+ MB


# Variables meaning

One of the first operation with the data is like when we meet a new person in the real life. We have to exchange some basic information. In that case we have already some information thanks to the metadata.

Columns are:
* `id`, Unique ID for the customer.
* `gender`, Gender of the customer.
* `age`, Age of the customer.
* `driving_license`, 0: Customer does not have DL, 1: Customer already has DL
* `region_code`, unique code for the region of the customer.
* `previously_insured`, 1: Customer already has Vehicle Insurance, 0: Customer doesn't have Vehicle Insurance.
* `vehicle_age`, age of the vehicle.
* `vehicle_damage`, 1: Customer got his/her vehicle damaged in the past. 0: Customer didn't get his/her vehicle damaged in the past.
* `annual_premium`, The amount customer needs to pay as premium in the year.
* `policy_sales_channel`, Anonymized Code for the channel of outreaching to the customer ie. Different Agents, Over Mail, Over Phone, In Person, etc.
* `vintage`, Number of Days, Customer has been associated with the company.
* `response`, 1: Customer is interested, 0: Customer is not interested.

Since I am italian I want to simulate the problem like the real scenario in Italy, where is not permitted to use the gender information in insurance companies for pricing and other reseach. Hence, I will drop that information. Moreover, I drop also the `id` since it could be useful only if we have other data to merge where there it is used as key. We do not have other data to merge and for prediction the `id` variable is useless.

In [3]:
train_df = input_df.copy()
train_df = train_df.drop(columns=['gender', 'id'])

# Target variable

Now it is time to inspect the target variable in the training data-set. The problem we are facing is a classification problem. Thanks to the traing data we need to build a model that is able to classify new records with the same kind of information (variables) in the correct way.

In [4]:
value_counts_response = train_df.response.value_counts().to_dict()

# Compute percentage of target classes
perc_label_0 = value_counts_response[0]/len(train_df)
perc_label_1 = value_counts_response[1]/len(train_df)
print(f"Percentage of records with response=0: {round(perc_label_0, 2)*100}%")
print(f"Percentage of records with response=1: {round(perc_label_1, 2)*100}%")

Percentage of records with response=0: 88.0%
Percentage of records with response=1: 12.0%


## Imbalanced data-set

As we can see there are much more records with `response=0`. That scenario in Machine Learning (aka ML) jargon is called _imbalanced data_. In the book _Introduction to Machine Learning with Python_, written by Andreas C. Müller and Sarah Guido they talk about the problem of imbalanced datasets.

_«Types of errors play an important role when one of two classes is much more frequent than the other one._

_This is very common in practice; a good example is click-through prediction, where each data point represents an “impression,” an item that was shown to a user. This item might be an ad, or a related story, or a related person to follow on a social media site._

_The goal is to predict whether, if shown a particular item, a user will click on it (indicating they are interested). Most things users are shown on the Internet (in particular, ads) will not result in a click. You might need to show a user 100 ads or articles before they find something interesting enough to click on._

_This results in a dataset where for each 99 “no click” data points, there is 1 “clicked” data point; in other words, 99% of the samples belong to the “no click” class. Datasets in which one class is much more frequent than the other are often called imbalanced datasets, or datasets with imbalanced classes._

_In reality, imbalanced data is the norm, and it is rare that the events of interest have equal or even similar frequency in the data.»_

Hence we have to use some algorithms to manage that kind of errors and create a balanced data-set. There are differnt methods to deal with imbalanced datasets. Those methods can be splitted into two main macro categories:
1. Under-sampling, we **reduce** the dimension of the imbalanced dataset removing records that has the label that appears most of the time. That famility of methods is useful when we have a huge amount of data and we can lose some information.
2. Over-sampling, we **increase** the dimension of the imbalanced dataset adding new records. That family of methods is useful since we do not lose original information. However, depending on the algorithm we decide to use we can add more or less noise and/or errros in the data.

The method I decide to use in this notebook is the _Synthetic Minority Over-sampling Technique for Nominal and Continuous_ (aka SMOTENC) implemented in the Python library `imbalanced-learn`. I decided to use it since I have both categorical (Nominal) and continuous variables. To further discussion on the over-sampling method that are implemented in the Python library `imbalanced-learn` click on the following [link](https://imbalanced-learn.org/stable/over_sampling.html#smote-adasyn).

# Experiment methods

To develop the experiment I want to build 3 different models based on the same algorithm `XGBClassifier`:
1. Train without parameters to balance data.
2. Train with parameter to balance data. The parameter `scale_pos_weight`, control the balance of positive and negative weights, useful for unbalanced classes. A typical value to consider: `sum(negative instances) / sum(positive instances)`. 
3. Train the model with balanced data using `SMOTENC` method to oversampling data.

## Pre-processing

The preprocessing step is necessary to modify values of the variables in order to be use from the model. The training data-set has continuous variables and categorical variables.

For the continuous variables I use the `StandardScaler` method. Instead, for the categorial variables we use the `OrdinalEncoder` for variables with a large set of possible values and the `OneHotEncoder` for other ones.

In [30]:
# Continuous variables
CONTINUOUS_VARS = ['age', 'annual_premium', 'vintage']
# Ordinal encoding variables
ORD_CATEGO_VARS = ['driving_license', 'region_code', 'previously_insured',
                          'policy_sales_channel']
# One-Hot Encoding variables
OHE_CATEGO_VARS = ['vehicle_age', 'vehicle_damage']
CATEGORICAL_VARS = ORD_CATEGO_VARS + OHE_CATEGO_VARS

# Change the dtype of categorical variables
for cat_var in CATEGORICAL_VARS:
    train_df[cat_var] = train_df[cat_var].astype('category')

# Define pre-processing methods
std_scl = StandardScaler()
ord_enc = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=np.nan)
ohe_enc = OneHotEncoder(handle_unknown='infrequent_if_exist')

preprocessor = ColumnTransformer(transformers=[
    ('num_enc', std_scl, CONTINUOUS_VARS),
    ('ord_enc', ord_enc, ORD_CATEGO_VARS),
    ('ohe_enc', ohe_enc, OHE_CATEGO_VARS)]
)

# Pipeline for experiment 1
pipe_1 = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', XGBClassifier(tree_method="hist",
                          enable_categorical=True))
])

# Compute scale_pos_weight value
value_counts_response = train_df.response.value_counts().to_dict()
scale_value = value_counts_response[0]/value_counts_response[1]

# Pipeline for experiment 2
pipe_2 = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', XGBClassifier(tree_method="hist",
                          enable_categorical=True,
                          scale_pos_weight=scale_value))
])

# Pipeline for experiment 3
pipe_3 = Pipeline(steps=[
    ('preprocess', preprocessor),
    ('clf', XGBClassifier(tree_method="hist",
                          enable_categorical=True))
])

In [12]:
# Create training and validation with unbalanced data
X_unb = train_df.copy()
y_unb = X_unb.pop('response')

X_unb_train, X_unb_test, y_unb_train, y_unb_test = train_test_split(X_unb,
                                                                    y_unb,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    random_state=1919)

# Create training and validation with over-sampled data
smote_nc = SMOTENC(categorical_features=CATEGORICAL_VARS,
                   random_state=1919)

X_res, y_res = smote_nc.fit_resample(X_unb, y_unb)
X_res_train, X_res_test, y_res_train, y_res_test = train_test_split(X_res,
                                                                    y_res,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    random_state=1919)

## Grid search cross validation

The grid search is a method to find the best set of parameters of the model in an automic way. Here I want to use the same possible set of parameters for all the three different models.

The parameters I want to change are:
* `max_depth`, maximum depth of a tree. Increasing this value will make the model more complex and more likely to overfit. 0 indicates no limit on depth. Beware that XGBoost aggressively consumes memory when training a deep tree.
* `eta`, it corresponds to the learning rate. It is a value in the interval [0, 1]. High value reduce the overfitting of the final model. However, value that are too much high does not find the correct optimum of the solution.
* `gamma`, minimum loss reduction required to make a further partition on a leaf node of the tree. The larger `gamma` is, the more conservative the algorithm will be. In other word low gamma can allows to the algorithm to create deeper trees and to catch different insights from the data in the training, however we have to use a generalised model.
* `lambda`, L2 regularization term on weights. Increasing this value will make model more conservative.

In [15]:
def perform_gridsearch(pipeline, param_grid, X_train, y_train, X_test, y_test):
    grid = GridSearchCV(pipeline, param_grid=param_grid, cv=5)
    grid.fit(X_train, y_train)
    
    print("Best cross-validation accuracy: {:.2f}".format(grid.best_score_))
    print("Test set score: {:.2f}".format(grid.score(X_test, y_test)))
    print("Best parameters: {}".format(grid.best_params_))

# We use the same parameters for all the three models
parameters_grid = {'clf__max_depth': [3, 8, 12, 15],
                   'clf__eta': [0.1, 0.15, 0.2, 0.3],
                   'clf__gamma': [0, 2, 5],
                   'clf__lambda': [1, 2, 3, 4]}

In [16]:
perform_gridsearch(pipeline=pipe_1,
                   param_grid=parameters_grid,
                   X_train=X_unb_train,
                   y_train=y_unb_train,
                   X_test=X_unb_test,
                   y_test=y_unb_test)

Best cross-validation accuracy: 0.88
Test set score: 0.88
Best parameters: {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 3, 'clf__max_depth': 3}


In [31]:
perform_gridsearch(pipeline=pipe_2,
                   param_grid=parameters_grid,
                   X_train=X_unb_train,
                   y_train=y_unb_train,
                   X_test=X_unb_test,
                   y_test=y_unb_test)

Best cross-validation accuracy: 0.80
Test set score: 0.80
Best parameters: {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 1, 'clf__max_depth': 15}


In [18]:
perform_gridsearch(pipeline=pipe_3,
                   param_grid=parameters_grid,
                   X_train=X_res_train,
                   y_train=y_res_train,
                   X_test=X_res_test,
                   y_test=y_res_test)

Best cross-validation accuracy: 0.86
Test set score: 0.86
Best parameters: {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 1, 'clf__max_depth': 15}


In [40]:
# pipe_1_best_param = {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 3, 'clf__max_depth': 3}
# pipe_2_best_param = {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 1, 'clf__max_depth': 15}
# pipe_3_best_param = {'clf__eta': 0.3, 'clf__gamma': 0, 'clf__lambda': 1, 'clf__max_depth': 15}

pipe_1_best = pipe_1.set_params(clf__eta=0.3,
                                clf__gamma=0,
                                clf__lambda=3,
                                clf__max_depth=3)
clf_model_1 = pipe_1_best.fit(X_unb_train, y_unb_train)
y_hat_1 = clf_model_1.predict(X_unb_test)

pipe_2_best = pipe_2.set_params(clf__eta=0.3,
                                clf__gamma=0,
                                clf__lambda=1,
                                clf__max_depth=15)
clf_model_2 = pipe_2_best.fit(X_unb_train, y_unb_train)
y_hat_2 = clf_model_2.predict(X_unb_test)

pipe_3_best = pipe_3.set_params(clf__eta=0.3,
                                clf__gamma=0,
                                clf__lambda=1,
                                clf__max_depth=15)
clf_model_3 = pipe_3_best.fit(X_res_train, y_res_train)
y_hat_3 = clf_model_3.predict(X_res_test)

In [41]:
print(classification_report(y_unb_test, y_hat_1))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93     66809
           1       0.50      0.01      0.01      9413

    accuracy                           0.88     76222
   macro avg       0.69      0.50      0.47     76222
weighted avg       0.83      0.88      0.82     76222



In [42]:
print(classification_report(y_unb_test, y_hat_2))

              precision    recall  f1-score   support

           0       0.93      0.83      0.88     66809
           1       0.32      0.57      0.41      9413

    accuracy                           0.80     76222
   macro avg       0.63      0.70      0.64     76222
weighted avg       0.86      0.80      0.82     76222



In [43]:
print(classification_report(y_res_test, y_hat_3))

              precision    recall  f1-score   support

           0       0.92      0.80      0.85     66953
           1       0.82      0.93      0.87     66807

    accuracy                           0.86    133760
   macro avg       0.87      0.86      0.86    133760
weighted avg       0.87      0.86      0.86    133760



# Conclusions

Overall, `model_1` is the one that has the highest accuracy. In particular, it has the highest value of F-1 score for the class `0`. However, it is not able to predict samples of class `1`. That is the main problem of unbalanced data-sets.

`model_2` has a lower accuracy respect to `model_1`, but it has an higher precision and recall for class `1`. Thus, it has a higher value of F-1 score. That means that without generating new data points the model is able to catch patterns also for class `1`.

The last model, `model_3` uses the over-sampled data-set. It reach the highest value of F-1 score for class `1`. Hower, it has a lower accuracy respect to the `model_1`. That is due to new generated data points that amplify the noise. However, in the use case it could be more relevant to discover clients that are interested to acquarie the new policy. When we find clients that are interested into the new policy the incsurance company has to apply some marketing strategies to reach suach a client.